# Manage Collections

> 本指南指导您使用所选的SDK创建和管理集合。

## 在开始之前

* 您已经安装了Milvus单机版或Milvus集群版。
* 您已安装首选SDK。您可以选择各种语言，包括Python，Java，Go和Node.js。

## 概述

在Milvus中，您将向量嵌入存储在集合中。集合中的所有向量嵌入共享相同的维度和距离度量来测量相似性。

Milvus集合支持动态字段（即，模式中未预定义的字段）和主键的自动递增。

为了适应不同的偏好，Milvus提供了两种创建集合的方法。一个提供快速设置，而另一个允许对集合模式和索引参数进行详细的自定义。

此外，您可以在必要时查看、加载、释放和删除集合。

此页面上的代码片段使用新的MilvusClient（Python）与Milvus交互。其他语言的新MilvusClient SDK将在未来的更新中发布。

## 创建集合


您可以通过以下任一方式创建集合：

> 快速设置

通过这种方式，您可以创建一个集合，只需给它一个名称并指定要存储在该集合中的向量嵌入的维数。有关详细信息，请参阅快速设置。

> 自定义设置

您可以自己决定集合的模式和索引参数，而不是让In Milvus决定集合的几乎所有内容。有关详细信息，请参阅自定义设置。

### 快速设置

在人工智能行业大飞跃的背景下，大多数开发人员只需要一个简单而动态的集合就可以开始。Milvus允许使用三个参数快速设置这样的集合：

* 要创建的集合的名称，
* 要插入的向量嵌入的维数，以及
* 度量类型，用于度量向量嵌入之间的相似性。

In [ ]:
from pymilvus import MilvusClient, DataType

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)

collection_name = "quick_setup"
if client.has_collection(collection_name):
    client.drop_collection(collection_name)

# 2. Create a collection in quick setup mode
client.create_collection(
    collection_name=collection_name,
    dimension=5
)

res = client.get_load_state(
    collection_name=collection_name
)

print(res)

```text
Output

{
    "state": "<LoadState: Loaded>"
}
```

在上面的代码中生成的集合只包含两个字段：id（作为主键）和vector（作为向量字段），默认情况下启用auto_id和enable_dynamic_field设置。

* `auto_id`: 启用此设置可确保主键自动递增。在数据插入过程中不需要手动提供主键。

* `enable_dynamic_field`: 启用后，除了要插入的数据中的id和vector之外，所有字段都被视为动态字段。这些附加字段作为键值对保存在名为$meta的特殊字段中。此功能允许在数据插入期间包含额外的字段。

从所提供的代码中自动索引和加载的集合已准备好立即插入数据。

### 自定义设置

您可以自己确定集合的模式和索引参数，而不是让Milvus决定集合的几乎所有内容。

#### 步骤1：设置架构

架构定义集合的结构。在模式中，您可以选择启用或禁用enable_dynamic_field、添加预定义字段以及为每个字段设置属性。有关概念和可用数据类型的详细解释，请参阅模式解释。

In [ ]:
# 3. Create a collection in customized setup mode

# 3.1. Create schema
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

# 3.2. Add fields to schema
schema.add_field(field_name="my_id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="my_vector", datatype=DataType.FLOAT_VECTOR, dim=5)

在提供的Python代码片段中，enable_dynamic_field被设置为True，并且为主键启用了auto_id。另外，引入了vector字段，其被配置为具有768的维度，沿着包括四个标量字段，每个标量字段具有其各自的属性。

#### 步骤2：设置索引参数

索引参数决定了Milvus如何在集合中组织数据。您可以通过调整特定字段的metric_type和index_type来定制其索引过程。对于向量场，您可以灵活地选择COSINE、L2或IP作为metric_type。

In [ ]:
# 3.3. Prepare index parameters
index_params = client.prepare_index_params()

# 3.4. Add indexes
index_params.add_index(
    field_name="my_id",
    index_type="STL_SORT"
)

index_params.add_index(
    field_name="my_vector", 
    index_type="IVF_FLAT",
    metric_type="IP",
    params={ "nlist": 128 }
)

上面的代码片段演示了如何分别为向量字段和标量字段设置索引参数。对于向量字段，设置度量类型和索引类型。对于标量字段，仅设置索引类型。建议为向量字段和任何经常用于筛选的标量字段创建索引。

#### 步骤3：创建集合

您可以选择分别创建集合和索引文件，或者创建一个集合，并在创建时同时加载索引。

创建一个集合，并在创建时同时加载索引。

In [ ]:
# 3.5. Create a collection with the index loaded simultaneously
client.create_collection(
    collection_name="customized_setup_1",
    schema=schema,
    index_params=index_params
)

# time.sleep(5)

res = client.get_load_state(
    collection_name="customized_setup_1"
)

print(res)

```text
# Output
#
# {
#     "state": "<LoadState: Loaded>"
# }
```

上面创建的集合将自动加载。若要了解有关加载和释放集合的更多信息，请参阅加载释放集合。

**分别创建集合和索引文件。**

In [ ]:
# 3.6. Create a collection and index it separately
client.create_collection(
    collection_name="customized_setup_2",
    schema=schema,
)

res = client.get_load_state(
    collection_name="customized_setup_2"
)

print(res)

```text
# Output
#
# {
#     "state": "<LoadState: NotLoad>"
# }
```

上面创建的集合不会自动加载。可以按如下方式为集合创建索引。以单独的方式为集合创建索引不会自动加载集合。有关详细信息，请参阅加载释放收集。&


In [ ]:
# 3.6 Create index
client.create_index(
    collection_name="customized_setup_2",
    index_params=index_params
)

res = client.get_load_state(
    collection_name="customized_setup_2"
)

print(res)

```text
# Output
#
# {
#     "state": "<LoadState: NotLoad>"
# }
```

## View Collections

In [ ]:
您可以按以下方式检查现有集合的详细信息：

In [ ]:
# 5. View Collections
res = client.describe_collection(
    collection_name="customized_setup_2"
)

print(res)

In [ ]:
要列出所有现有集合，您可以执行以下操作：

In [ ]:
# 6. List all collection names
res = client.list_collections()

print(res)

## 装载和释放收集

在加载集合的过程中，Milvus将集合的索引文件加载到内存中。相反，当释放一个集合时，Milvus会从内存中卸载索引文件。在集合中执行搜索之前，请确保该集合已加载。


### 加载集合

要加载集合，请使用load_collection()方法，指定集合名称。您还可以设置replica_number来确定加载集合时在查询节点上创建多少数据段的内存中副本。

- Milvus Standalone：replica_number允许的最大值为1。
- Milvus集群：最大值不应超过Milvus配置中设置的queryNode.replicas。有关其他详细信息，请参阅与查询节点相关的配置文件。

In [ ]:
# 7. Load the collection
client.load_collection(
    collection_name="customized_setup_2",
    replica_number=1 # Number of replicas to create on query nodes. Max value is 1 for Milvus Standalone, and no greater than `queryNode.replicas` for Milvus Cluster.
)

res = client.get_load_state(
    collection_name="customized_setup_2"
)

print(res)

### Release a collection

In [ ]:
# 8. Release the collection
client.release_collection(
    collection_name="customized_setup_2"
)

res = client.get_load_state(
    collection_name="customized_setup_2"
)

In [ ]:
print(res)

## 设置别名


您可以为集合分配别名，以使它们在特定上下文中更有意义。可以为一个集合分配多个别名，但多个集合不能共享一个别名。


### 创建别名

In [ ]:
# 9.1. Create aliases
client.create_alias(
    collection_name="customized_setup_2",
    alias="bob"
)

client.create_alias(
    collection_name="customized_setup_2",
    alias="alice"
)

### 列出别名

In [ ]:
# 9.2. List aliases
res = client.list_aliases(
    collection_name="customized_setup_2"
)

print(res)

### 描述别名

In [ ]:
# 9.3. Describe aliases
res = client.describe_alias(
    alias="bob"
)

print(res)

### 重新分配别名


In [ ]:
# 9.4 Reassign aliases to other collections
client.alter_alias(
    collection_name="customized_setup_1",
    alias="alice"
)

res = client.list_aliases(
    collection_name="customized_setup_1"
)

print(res)

In [ ]:
res = client.list_aliases(
    collection_name="customized_setup_2"
)

print(res)

### 删除别名

In [ ]:
# 9.5 Drop aliases
client.drop_alias(
    alias="bob"
)

client.drop_alias(
    alias="alice"
)

## Drop a Collection

In [ ]:
# 10. Drop the collections
client.drop_collection(
    collection_name="quick_setup"
)

client.drop_collection(
    collection_name="customized_setup_1"
)

client.drop_collection(
    collection_name="customized_setup_2"
)

-------